In [22]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, SGDRegressor, Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, StratifiedShuffleSplit,\
RandomizedSearchCV
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from hyperopt.pyll.stochastic import sample

In [2]:
#read in prepared dataset from EDA.ipynb notebook
mlb_df = pd.read_csv('model_ready_.csv.gz', compression = 'gzip')

In [3]:
pd.set_option('max.columns', 25)
mlb_df.head()

,pitcher2_rgs,pitcher1_rgs,rating_prob1,pitcher1_adj,pitcher2_adj,run_differential_hm,rating1_pre,run_differential_rd,rating2_pre,TMAX,attendance,pitching_park_factor,...,PRCP,current_streak_rd_tm,current_streak_hm_tm,avg_margin_rd,current_streak_hm_at_hm,avg_margin_hm,is_liveball,is_FA,is_daygame,is_integration,is_deadball,score_differential
0,-0.140582,-0.968011,-0.243000,-0.002938,-0.007462,-0.162090,-0.026615,0.132497,0.343735,-0.757834,-0.487611,-0.448150,...,0.0,0.230927,-0.026751,0.088201,-0.132420,-0.086008,0.0,0.0,1,0.0,1.0,0.669243
1,-0.710888,0.006553,0.252994,-0.003285,-0.007898,-0.162090,1.264020,0.132497,0.894793,-0.371693,-0.630211,2.415975,...,0.0,0.230927,-0.026751,0.088201,-0.132420,-0.086008,0.0,0.0,1,0.0,1.0,-0.508597
2,0.007053,-0.714184,-1.188872,-0.000870,-0.004514,-0.162090,-0.769264,0.132497,1.024317,-2.319949,-0.273710,-0.668467,...,0.0,0.230927,-0.026751,0.088201,-0.132420,-0.086008,0.0,0.0,1,0.0,1.0,-0.273029
3,0.006954,0.004049,0.267979,-0.001778,-0.004704,-0.162090,0.426835,0.132497,0.089025,-1.337045,-0.487611,-0.227833,...,0.0,0.230927,-0.026751,0.088201,-0.132420,-0.086008,0.0,0.0,1,0.0,1.0,-0.744165
4,0.004580,0.004351,-1.272155,-0.002941,-0.003425,-0.186952,-0.800985,0.157245,1.056050,-2.021568,-1.152412,-0.668467,...,0.0,0.685557,-0.437490,1.082324,-0.516648,-1.088560,0.0,0.0,1,0.0,1.0,0.904811


In [4]:
#isolate target feature
X, y = mlb_df.drop(columns = ['score_differential']), mlb_df['score_differential']
#split into train/test sets 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [5]:
#create RMSE metric to evaluate regression
def get_metrics(y_true, y_pred):
    RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
    EV = explained_variance_score(y_true, y_pred)
    R2 = r2_score(y_true, y_pred)
    return({'RMSE' : RMSE, 'explained_variance' : EV, 'r2_score' : R2})

In [17]:
#create baseline model to always predict mean
dummy_mean = DummyRegressor(strategy = 'mean')
#fit dummy regressor
dummy_mean.fit(x_train, y_train)
#predict on test set
dummy_pred = dummy_mean.predict(x_test)
#get metrics
baseline_metrics = get_metrics(y_test, dummy_pred)
for metric in baseline_metrics.keys():
    print('The {} for the baseline model is: {}.'.format(metric, baseline_metrics[metric]))

The RMSE for the baseline model is: 0.9935870965647374.
The explained_variance for the baseline model is: 0.0.
The r2_score for the baseline model is: -2.6410170201884853e-06.


In [18]:
#train basic linear regression model 
lm = LinearRegression()
lm.fit(x_train, y_train)
#predict on test set
lm_pred = lm.predict(x_test)
#get metrics
lm_metrics = get_metrics(y_test, lm_pred)
for metric in lm_metrics.keys():
    print('The {} for the Linear Regression model is {}.'.format(metric, lm_metrics[metric]))

The RMSE for the Linear Regression model is 0.9749089030246673.
The explained_variance for the Linear Regression model is 0.03724859402298397.
The r2_score for the Linear Regression model is 0.037241560607933266.


In [22]:
#create list of linear models to iterate through
algorithms = [
    ("L1_Regularizatiion", Lasso(selection = 'random', random_state = 42)),
    ("L2_Regularization", Ridge(solver = 'auto', random_state = 42)),
    ("Elastic_Net", ElasticNet(selection = 'random', random_state = 42)),
    ("SGDRegressor", SGDRegressor(loss = 'squared_loss', penalty = 'l2', l1_ratio = 0.2, random_state = 42))
]
#iterate through algorithms, evaluate metrics
eval_metrics = []
index_list = []
for name, algorithm in algorithms:
    model = algorithm.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    metrics = get_metrics(y_test, y_pred)
    eval_metrics.append(metrics)
    index_list.append(name)

idx = pd.Series(index_list)
eval_df = pd.DataFrame(eval_metrics)
#view results
pd.concat([idx, eval_df], axis = 1)

,0,RMSE,explained_variance,r2_score
0,L1_Regularizatiion,0.993587,0.000000,-0.000003
1,L2_Regularization,0.974909,0.037249,0.037242
2,Elastic_Net,0.993587,0.000000,-0.000003
3,SGDRegressor,0.976814,0.033483,0.033475


In [45]:
#create parameter space for XGBoost algorithm
def xgb_optimize(x_train, y_train, num_eval):
    xgb_space = {'objective' : 'reg:squarederror',
            'verbosity' : 0,
            'booster' : 'gbtree',
            'max_depth' : scope.int(hp.quniform('max_depth', 3, 10, 1)),
            'gamma' : hp.uniform('gamma', 0, 0.5),
            'n_estimators' : scope.int(hp.quniform('n_estimators', 100, 1000, 100)),
            'min_child_weight' : scope.int(hp.quniform('min_child_weight', 1, 6, 1)),
            'subsample' : hp.uniform('subsample', 0.5, 0.8),
            'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 0.8),
            'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
            'random_state' : 43
                }
    dtrain = xgb.DMatrix(x_train, label = y_train)
    
    def xgb_objective(params):
        xbr = xgb.XGBRegressor(**params)
        current_params = xbr.get_xgb_params()
        cv_results = xgb.cv(current_params, dtrain, num_boost_round = xbr.get_params()['n_estimators'],
                           nfold = 5, metrics = 'rmse', early_stopping_rounds = 50)
        score = cv_results.iloc[-1]['test-rmse-mean']
        return({'loss' : score, 'status' : STATUS_OK})
    
    trials = Trials()
    
    best_params = fmin(xgb_objective, xgb_space, algo = tpe.suggest, max_evals = num_eval, trials = trials)
    
    return(best_params)

In [46]:
best_params = xgb_optimize(x_train, y_train, 10)

100%|██████████| 10/10 [37:19<00:00, 223.96s/it, best loss: 0.9819866000000002]


In [47]:
best_params

{'colsample_bytree': 0.6194228466063548,
 'gamma': 0.3262944616419851,
 'learning_rate': 0.012090211212838623,
 'max_depth': 5.0,
 'min_child_weight': 2.0,
 'n_estimators': 700.0,
 'subsample': 0.7348817995646493}

In [9]:
#fit XGBRegressor using tuned parameters
XGR = xgb.XGBRegressor(objective = 'reg:squarederror',
                       colsample_bytree = 0.6, gamma = 0.3, learning_rate = 0.01, max_depth = 5,
                      min_child_weight = 2, n_estimators = 700, subsample = 0.75, random_state = 42)

XGR.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0.3,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=2, missing=None, n_estimators=700,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)

In [10]:
#generate predictions
y_preds = XGR.predict(x_test)

In [11]:
xgb_metrics = get_metrics(y_test, y_preds)

In [13]:
xgb_metrics

{'RMSE': 0.9746478808425186,
 'explained_variance': 0.037765028523061295,
 'r2_score': 0.03775702962506289}

Even though I did I great deal of EDA in the previous notebooks, these results are so bad I need to go back into the data. I will check the selected features for multicollinearity by using the Variance Inflation Factor.

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [11]:
#VIF calculation requires a column of constants to be added to the independent variable matrix. 
X_vif = add_constant(X.values)

In [12]:
#calculate the VIF for each column
VIF = pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)

In [13]:
VIF

pitcher2_rgs                    1.240140
pitcher1_rgs                    1.258849
rating_prob1                  508.310143
pitcher1_adj                   12.961734
pitcher2_adj                   12.877430
run_differential_hm             2.649079
rating1_pre                   230.612614
run_differential_rd             2.695146
rating2_pre                   230.257557
TMAX                            1.034679
attendance                      1.494466
pitching_park_factor            1.318674
elevation                       1.303063
distance_traveled               1.041233
current_streak_rd_tm_on_rd      2.450703
PRCP                            1.056936
current_streak_rd_tm            2.270183
current_streak_hm_tm            3.189502
avg_margin_rd                   1.691734
current_streak_hm_at_hm         3.154301
avg_margin_hm                   1.671766
is_liveball                     1.845399
is_FA                           1.128220
is_daygame                      2.591353
is_integration  

In [14]:
#return list of features where VIF is alarmingly high
problem_VIF = list(VIF[VIF.values > 10].index)

In [15]:
problem_VIF

['rating_prob1', 'pitcher1_adj', 'pitcher2_adj', 'rating1_pre', 'rating2_pre']

In [16]:
X_red = X.drop(columns = problem_VIF)

In [17]:
x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_red, y, test_size = 0.2, random_state = 43)

In [24]:
#train basic linear regression model 
lm = LinearRegression()
lm.fit(x_train_n, y_train_n)
#predict on test set
lm_pred = lm.predict(x_test_n)
#get metrics
lm_metrics = get_metrics(y_test_n, lm_pred)
for metric in lm_metrics.keys():
    print('The {} for the Linear Regression model is {}.'.format(metric, lm_metrics[metric]))

The RMSE for the Linear Regression model is 0.9805785877290607.
The explained_variance for the Linear Regression model is 0.027930291477181513.
The r2_score for the Linear Regression model is 0.027911809012416855.


In [25]:
#create list of linear models to iterate through
algorithms = [
    ("L1_Regularizatiion", Lasso(selection = 'random', random_state = 42)),
    ("L2_Regularization", Ridge(solver = 'auto', random_state = 42)),
    ("Elastic_Net", ElasticNet(selection = 'random', random_state = 42)),
    ("SGDRegressor", SGDRegressor(loss = 'squared_loss', penalty = 'l2', l1_ratio = 0.2, random_state = 42))
]
#iterate through algorithms, evaluate metrics
eval_metrics = []
index_list = []
for name, algorithm in algorithms:
    model = algorithm.fit(x_train_n, y_train_n)
    y_pred = model.predict(x_test_n)
    metrics = get_metrics(y_test_n, y_pred)
    eval_metrics.append(metrics)
    index_list.append(name)

idx = pd.Series(index_list)
eval_df = pd.DataFrame(eval_metrics)
#view results
pd.concat([idx, eval_df], axis = 1)

,0,RMSE,explained_variance,r2_score
0,L1_Regularizatiion,0.994566,0.000000,-0.000019
1,L2_Regularization,0.980579,0.027930,0.027912
2,Elastic_Net,0.994566,0.000000,-0.000019
3,SGDRegressor,0.981472,0.026283,0.026140


In [6]:
xbr = xgb.XGBRegressor(objective = 'reg:squarederror')

In [7]:
xbr.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [8]:
y_preds = xbr.predict(x_test)

In [9]:
get_metrics(y_test, y_preds)

{'RMSE': 0.9748428028749276,
 'explained_variance': 0.037377413957349725,
 'r2_score': 0.03737210884566455}

In [20]:
xbr = xgb.XGBRegressor(objective = 'reg:squarederror')
xbr.fit(x_train_n, y_train_n)
y_preds = xbr.predict(x_test_n)
get_metrics(y_test_n, y_preds)


{'RMSE': 0.9807378652784794,
 'explained_variance': 0.02761727874242703,
 'r2_score': 0.0275959864936407}

In [23]:
XBR = xgb.XGBRegressor(objective = 'reg:squarederror')

In [24]:
XBR.get_xgb_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'importance_type': 'gain',
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': 1,
 'objective': 'reg:squarederror',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'subsample': 1,
 'verbosity': 1}

In [25]:
param_grid = {'colsample_bytree' : [0.5, 0.75, 1.0],
             'max_depth' : [3, 5, 7, 9],
             'min_child_weight' : [0.5, 1.0],
             'subsample' : [0.5, 0.75, 1.0]}
random_search = RandomizedSearchCV(XBR, param_grid, random_state = 43)

In [26]:
XGTrain = xgb.DMatrix(x_train.values, label = y_train.values)

In [30]:
x_train_np = np.array(x_train)
y_train_np = np.array(y_train)
x_test_np = np.array(x_test)
y_test_np = np.array(y_test)

In [31]:
search = random_search.fit(x_train_np, y_train_np)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [33]:
pd.DataFrame(search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_min_child_weight,param_max_depth,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,28.399510,0.190487,0.248880,0.003141,0.5,1,5,1,"{'subsample': 0.5, 'min_child_weight': 1.0, 'm...",0.035921,0.035030,0.035621,0.035524,0.000370,4
1,40.649747,0.193675,0.429709,0.008081,0.5,1,7,1,"{'subsample': 0.5, 'min_child_weight': 1.0, 'm...",0.031223,0.028803,0.028881,0.029636,0.001123,8
2,56.094304,0.677879,0.584984,0.007267,0.75,1,9,1,"{'subsample': 0.75, 'min_child_weight': 1.0, '...",0.026213,0.024002,0.024780,0.024998,0.000916,10
3,19.413177,0.432719,0.165132,0.003543,0.75,1,3,1,"{'subsample': 0.75, 'min_child_weight': 1.0, '...",0.038689,0.037563,0.038038,0.038096,0.000461,3
4,26.458323,0.133916,0.423393,0.004703,0.75,0.5,7,0.5,"{'subsample': 0.75, 'min_child_weight': 0.5, '...",0.034889,0.032286,0.032783,0.033320,0.001128,6
5,25.563272,0.224323,0.446083,0.008649,0.5,1,7,0.5,"{'subsample': 0.5, 'min_child_weight': 1.0, 'm...",0.029829,0.031482,0.031395,0.030902,0.000760,7
6,13.686309,0.120636,0.159069,0.004216,1,1,3,0.5,"{'subsample': 1.0, 'min_child_weight': 1.0, 'm...",0.038732,0.037695,0.037952,0.038126,0.000441,2
7,44.181541,0.310536,0.381347,0.009082,1,0.5,7,1,"{'subsample': 1.0, 'min_child_weight': 0.5, 'm...",0.033983,0.033566,0.033047,0.033532,0.000383,5
8,44.113638,0.408194,0.524187,0.008761,1,0.5,9,0.75,"{'subsample': 1.0, 'min_child_weight': 0.5, 'm...",0.030301,0.028418,0.029012,0.029244,0.000786,9
9,16.751789,0.298909,0.161895,0.001903,1,0.5,3,0.75,"{'subsample': 1.0, 'min_child_weight': 0.5, 'm...",0.038727,0.037846,0.038114,0.038229,0.000369,1


In [34]:
search.best_params_

{'subsample': 1.0,
 'min_child_weight': 0.5,
 'max_depth': 3,
 'colsample_bytree': 0.75}